<H1> Load Toronto Dataset </H1>

In [1]:
import pandas as pd
TorontoDf = pd.read_excel("FinalTorontoNeighborhood.xlsx")
TorontoDf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<H2> Extracting Boroughs with Toronto in it </H2>

In [8]:
# We will only analyse data from Toronto Boroughs

# Extracting all rows, where Borough has 'Toronto'
toronto_borough = TorontoDf[TorontoDf['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_borough.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


<H2> Foursquare Api Credentials </H2>

In [10]:
#@hidden_cell
CLIENT_ID = 'QUC1AD5AAIO4Y5Q54CBZQWVBQKLNLG1N5GLKMOH2SV4VKMPC' # your Foursquare ID
CLIENT_SECRET = 'AJWL0IN0XSFF440CYNK2CCQF1ZKLDDA15OUAHZ5VVOKB4XPD' # your Foursquare Secret
VERSION = '20190815'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QUC1AD5AAIO4Y5Q54CBZQWVBQKLNLG1N5GLKMOH2SV4VKMPC
CLIENT_SECRET:AJWL0IN0XSFF440CYNK2CCQF1ZKLDDA15OUAHZ5VVOKB4XPD


<H3> Extracting venues information from Toronto neighborhoods </H3>

In [18]:
# Function to extract all venuse information from neighborhoods
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=5
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
# Extracting venues information for boroughs - Toronto
borough_toronto = getNearbyVenues(names=toronto_borough['Neighborhood'],
                                   latitudes=toronto_borough['Latitude'],
                                   longitudes=toronto_borough['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [20]:
borough_toronto.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop
9,"The Beaches West, India Bazaar",43.668999,-79.315572,British Style Fish & Chips,43.668723,-79.317139,Fish & Chips Shop


<H3> Analyse each neighborhood </H3>

In [38]:
# one hot encoding
borough_toronto_onehot = pd.get_dummies(borough_toronto[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
borough_toronto_onehot['Neighborhoods'] = borough_toronto['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [borough_toronto_onehot.columns[-1]] + list(borough_toronto_onehot.columns[:-1])
#print(fixed_columns)
borough_toronto_onehot = borough_toronto_onehot[fixed_columns]

borough_toronto_onehot.head()

,Neighborhoods,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,...,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<H2> Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.
Let's print each neighborhood along with the top 5 most common venues</H2>

In [40]:
borough_toronto_onehot_grouped = borough_toronto_onehot.groupby('Neighborhoods').mean().reset_index()
num_top_venues = 5

for hood in borough_toronto_onehot_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = borough_toronto_onehot_grouped[borough_toronto_onehot_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                           venue  freq
0                     Steakhouse   0.2
1  Vegetarian / Vegan Restaurant   0.2
2                   Concert Hall   0.2
3                          Plaza   0.2
4                          Hotel   0.2


----Berczy Park----
                           venue  freq
0                     Steakhouse   0.2
1  Vegetarian / Vegan Restaurant   0.2
2                   Concert Hall   0.2
3                         Museum   0.2
4                   Liquor Store   0.2


----Brockton, Exhibition Place, Parkdale Village----
                  venue  freq
0           Coffee Shop   0.4
1                   Bar   0.2
2  Caribbean Restaurant   0.2
3                   Gym   0.2
4                 Hotel   0.0


----Business Reply Mail Processing Centre 969 Eastern----
            venue  freq
0  Farmers Market   0.2
1      Comic Shop   0.2
2   Burrito Place   0.2
3         Brewery   0.2
4     Pizza Place   0.2


----CN Tower, Bathurst Quay, Island 

<H3> Storing above information in Pandas data frame </H3>

In [41]:
# A function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
# create the new dataframe and display the top 5 venues for each neighborhood
import numpy as np
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = borough_toronto_onehot_grouped['Neighborhoods']

for ind in np.arange(borough_toronto_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_toronto_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Vegetarian / Vegan Restaurant,Plaza,Concert Hall,Hotel,Steakhouse
1,Berczy Park,Museum,Vegetarian / Vegan Restaurant,Liquor Store,Concert Hall,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Caribbean Restaurant,Gym,Bar,Dessert Shop
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Comic Shop,Burrito Place,Farmers Market,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport,Airport Food Court,Airport Lounge,Airport Terminal,Harbor / Marina


<H1> Clustering the neighborhoods </H1>


In [47]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

borough_toronto_clustering = borough_toronto_onehot_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(borough_toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 1, 1, 1, 1, 1])

In [53]:
# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_borough
toronto_merged=toronto_merged.rename(columns = {'Neighborhood':'Neighborhoods'})
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Neighborhood,Trail,Pub,Health Food Store,Cuban Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Ice Cream Shop,Greek Restaurant,Cosmetics Shop,Italian Restaurant,Fish Market
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Gym,Fish & Chips Shop,Burger Joint,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Bookstore,Sandwich Place,Coffee Shop,Fish Market,Ice Cream Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Dance Studio,Cocktail Bar


<H2> Generate clusters on maps </H2>

In [57]:
# Extract Toronto Latitude and Longitude
from geopy.geocoders import Nominatim 

address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("Toronto !!")
print(latitude, longitude)



Toronto !!
43.653963 -79.387207


In [60]:
# Generating map. Highlighting neighborhoods and clusters for Toronto

import folium
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhoods'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters